### Setup: Install and imports

In [1]:
!pip install numpy
!pip install ogb==1.2.4
!pip install tqdm
!pip install hyperopt
!pip install ray==2.3.0
!pip install torch

!pip install deap
!pip install -U tensorboardx

In [2]:
import os
from tqdm import tqdm
import numpy as np
from hyperopt import hp
from hyperopt import tpe
from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch
from ray.air.config import RunConfig
from ogb.linkproppred import LinkPropPredDataset, Evaluator

from google.colab import drive

from deap import base, creator, tools, algorithms

In [3]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Dataset preparation:

Load the rank file for different datasets:
1.   obgl-biokg (KGBench, ComplexRP, TripleRE)
2.   ogbl-wikikg2 (Text, InterHTPlus, StarGraph)



In [4]:
dataset_name = 'ogbl-biokg' # ogbl-biokg, ogbl-wikikg2

rank_path = f"./gdrive/MyDrive/CPSC_583_Dataset/{dataset_name}"
checkpoint_path = f"weights/{dataset_name}_rel_weights.npy"

max_concurrent_trials = 8
num_samples = 100
num_initial_points = 20

In [19]:
evaluator = Evaluator(name=dataset_name)

if dataset_name == 'ogbl-biokg':
    model_names = ['rescal', 'rotate', 'cp', 'complex', 'unibi_3', 'triplere'] #, 'transe', 'tucker']
else:
    raise NotImplementedError(f"Unsupported dataset: {dataset_name}")

In [20]:
print(f"Loading dataset: {dataset_name}")
ranks = {
    'valid': [np.load(f"{rank_path}/{m}_valid_ranks.npy") for m in model_names],
    'test': [np.load(f"{rank_path}/{m}_test_ranks.npy") for m in model_names]
}
print(ranks['test'][0].shape)

n_model = len(ranks['test'])

Loading dataset: ogbl-biokg
(162870, 1002)


In [21]:
dataset = LinkPropPredDataset(name=dataset_name)
split_edge = dataset.get_edge_split()
train_triples, valid_triples, test_triples = split_edge["train"], split_edge["valid"], split_edge["test"]

In [22]:
if dataset_name == 'ogbl-biokg':
    test_relation = test_triples['relation']
    valid_relation = valid_triples['relation']
    num_relation = int(max(train_triples['relation']))+1
elif dataset_name == 'ogbl-wikikg2':
    origin_num_relation = int(max(train_triples['relation'].max(), valid_triples['relation'].max(), test_triples['relation'].max()))+1
    test_relation = np.concatenate((test_triples['relation'], test_triples['relation'] + origin_num_relation), axis=0)
    valid_relation = np.concatenate((valid_triples['relation'], valid_triples['relation'] + origin_num_relation), axis=0)
    num_relation = int(max(test_relation.max(), valid_relation.max())) + 1

print(num_relation)

51


In [23]:
rel_indexes = {
    'valid': {},
    'test': {}
} # relation_id -> np array

for relation_id in range(num_relation):
    rel_indexes['test'][relation_id] = np.where(test_relation == relation_id)[0]
    rel_indexes['valid'][relation_id] = np.where(valid_relation == relation_id)[0]


### Utility functions

In [24]:
def eval_model(sub_ranks):
    new_ranks = 502 - sub_ranks

    mrr_head = evaluator.eval({'y_pred_pos': new_ranks[:, 0], 'y_pred_neg': new_ranks[:, 1:501]})['mrr_list'].mean()
    mrr_tail = evaluator.eval({'y_pred_pos': new_ranks[:, 501], 'y_pred_neg': new_ranks[:, 502:]})['mrr_list'].mean()

    return {'mrr': (mrr_head + mrr_tail) / 2}

In [25]:
 # Modification
def adjust_weights_simple(current_weights, performance_metrics, adjustment_factor=0.1):
    """
    Adjust the ensemble weights based on performance metrics.
    Here, a simple strategy is used: increase the weight of better-performing models
    by a factor and decrease the weight of others. This is a basic example and
    can be replaced with more sophisticated methods.
    """
    best_model_index = np.argmax(performance_metrics)
    for i in range(len(current_weights)):
        if i == best_model_index:
            current_weights[i] += adjustment_factor
        else:
            current_weights[i] -= adjustment_factor / (len(current_weights) - 1)
    # Ensure weights remain normalized
    current_weights = np.maximum(current_weights, 0)
    current_weights /= np.sum(current_weights)
    return current_weights

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def adjust_weights(current_weights, gradient, learning_rate=0.01, momentum=0.9):
    """
    Adjust weights using a gradient-based approach with a learning rate and momentum.
    """
    gradient = np.array(gradient)
    previous_update = np.array(adjust_weights.previous_update)

    update = momentum * previous_update - learning_rate * gradient
    new_weights = current_weights + update
    # Normalize with softmax
    new_weights = softmax(new_weights)
    adjust_weights.previous_update = update
    return new_weights

In [26]:
def objective(config, data, dynamic=False, current_weights=None):
    sub_ranks = data
    weights = np.array([config[f"w_{i}"] for i in range(len(sub_ranks))])
    ranks_avg = np.average(sub_ranks, weights=weights, axis=0)
    mrr = eval_model(ranks_avg)

    if not dynamic:
        return mrr

    gradient = np.array([mrr['mrr'] for _ in range(len(sub_ranks))])
    new_weights = adjust_weights(current_weights, gradient, learning_rate, momentum)
    return mrr, new_weights

### Machine learning ensemble

1. Default Configuration Setup
2. Loading or Initializing Ensemble Weights
3. Defining the Search Space for Hyperparameter Optimization:
4. Setting Up and Running the Optimization Process:
5. Potential Saving of Computed Weights:






In [27]:
momentum = 0.9
learning_rate = 0.01
dynamic_weight_adjust = True
dynamic_iterations = 16 if dynamic_weight_adjust else 1

default_config = {
    'w_0': 0.16,
    'w_1': 0.16,
    'w_2': 0.16,
    'w_3': 0.17,
    'w_4': 0.17,
    'w_5': 0.17,
}

initial_weights = np.array([default_config['w_0'], default_config['w_1'], default_config['w_2'], default_config['w_3'], default_config['w_4'], default_config['w_5']])
adjust_weights.previous_update = np.zeros(len(initial_weights))

if checkpoint_path and os.path.exists(checkpoint_path):
    print("Load existing models")
    rel_weights = np.load(checkpoint_path)
else:
    print("Searching for ensemble weights")
    rel_weights = np.zeros((num_relation, n_model))

    search_space = {f"w_{i}": hp.uniform(f"w_{i}", 0, 1)  for i in range(n_model)}
    hyperopt_search = HyperOptSearch(search_space, metric="mrr", mode="max", n_initial_points=num_initial_points)

    for rel_id in tqdm(range(num_relation)):
        if len(rel_indexes['valid'][rel_id]) == 0:
            # default weights
            rel_weights[rel_id] = np.fromiter(default_config.values(), dtype=np.float32)
            continue

        subranks = [model_rank[rel_indexes['valid'][rel_id]] for model_rank in ranks['valid']]
        tuner = tune.Tuner(tune.with_parameters(objective, data=(subranks)), param_space=search_space,
                tune_config=tune.TuneConfig(num_samples=num_samples, search_alg=hyperopt_search, max_concurrent_trials=max_concurrent_trials),
                run_config=RunConfig(verbose=0))
        results = tuner.fit()

        best_weights = np.fromiter(results.get_best_result(metric="mrr", mode="max").config.values(), dtype='float32')

        if dynamic_weight_adjust:
            # Dynamic weight adjustment
            current_weights = best_weights
            for _ in range(dynamic_iterations):
                mrr, current_weights = objective({'w_0': current_weights[0], 'w_1': current_weights[1], 'w_2': current_weights[2], 'w_3': current_weights[3], 'w_4': current_weights[4], 'w_5': current_weights[5]}, subranks, dynamic_weight_adjust, current_weights=current_weights)

            rel_weights[rel_id] = current_weights
        else:
            rel_weights[rel_id] = best_weights

    # np.save(f"rel_weights.npy", rel_weights)


(pid=478376) WARNING:root:The OGB package is out of date. Your version is 1.2.4, while the latest version is 1.3.6.
(pid=478390) WARNING:root:The OGB package is out of date. Your version is 1.2.4, while the latest version is 1.3.6.
(pid=478453) WARNING:root:The OGB package is out of date. Your version is 1.2.4, while the latest version is 1.3.6.
(pid=478490) WARNING:root:The OGB package is out of date. Your version is 1.2.4, while the latest version is 1.3.6.
(pid=478512) WARNING:root:The OGB package is out of date. Your version is 1.2.4, while the latest version is 1.3.6.
(pid=478618) WARNING:root:The OGB package is out of date. Your version is 1.2.4, while the latest version is 1.3.6.
(pid=478581) WARNING:root:The OGB package is out of date. Your version is 1.2.4, while the latest version is 1.3.6.
(pid=478709) WARNING:root:The OGB package is out of date. Your version is 1.2.4, while the latest version is 1.3.6.
2023-12-13 04:59:50,148	WARNING util.py:244 -- The `callbacks.on_trial_r

In [18]:
print("Evaluating")
rel_res = {
    'test': [{'mrr': 0} for _ in range(num_relation)],
    'valid': [{'mrr': 0} for _ in range(num_relation)]
}

for rel_id in tqdm(range(num_relation)):
    # test results
    if len(rel_indexes['test'][rel_id]) == 0:
        continue
    sub_ranks = [model_rank[rel_indexes['test'][rel_id]] for model_rank in ranks['test']]
    config = {f"w_{i}": rel_weights[rel_id][i] for i in range(n_model)}
    metrics = objective(config, (sub_ranks))
    rel_res['test'][rel_id] = metrics

    # valid results
    if len(rel_indexes['valid'][rel_id]) == 0:
        continue
    sub_ranks = [model_rank[rel_indexes['valid'][rel_id]] for model_rank in ranks['valid']]
    config = {f"w_{i}": rel_weights[rel_id][i] for i in range(n_model)}
    metrics = objective(config, (sub_ranks))
    rel_res['valid'][rel_id] = metrics

test_mrr = 0
valid_mrr = 0

for rel_id in range(num_relation):
    test_mrr += rel_res['test'][rel_id]['mrr'] * len(rel_indexes['test'][rel_id])
    valid_mrr += rel_res['valid'][rel_id]['mrr'] * len(rel_indexes['valid'][rel_id])

test_mrr = test_mrr / ranks['test'][0].shape[0]
valid_mrr = valid_mrr / ranks['valid'][0].shape[0]

print(f"\nTest MRR: {test_mrr}\nValidation MRR: {valid_mrr}")

Evaluating
 100%|██████████| 51/51 [00:30<00:00,  1.66it/s]
 Test MRR: 0.7873165219712932
 Validation MRR: 0.3981841449958312



Genetic Algorithm

In [28]:
# Define the fitness function
def evalModel(individual):
    weights = np.array(individual)
    normalized_weights = weights / np.sum(weights)

    # Compute the weighted average ranks across all relations
    all_mrrs = []
    for rel_id in range(num_relation):
        if len(rel_indexes['valid'][rel_id]) > 0:
            sub_ranks = [model_rank[rel_indexes['valid'][rel_id]] for model_rank in ranks['valid']]
            ranks_avg = np.average(sub_ranks, weights=normalized_weights, axis=0)
            mrr = eval_model(ranks_avg)
            all_mrrs.append(mrr['mrr'])

    # Average MRR across all relations
    avg_mrr = np.mean(all_mrrs) if all_mrrs else 0

    return avg_mrr,

# Set up the GA
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=n_model)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evalModel)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# GA parameters
population_size = 50
number_of_generations = 50

# Initialize population
population = toolbox.population(n=population_size)

# Run the GA
for gen in range(number_of_generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.2)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
    best_ind = tools.selBest(population, k=1)[0]
    #print(f"Generation {gen}: Best MRR: {best_ind.fitness.values[0]}")

top_individual = tools.selBest(population, k=1)[0]
print(f"Best MRR: {top_individual.fitness.values[0]}")

Best MRR: 0.652365818047056
